In [12]:
import sys
sys.path.append('../utils/')
from utils import *
from sap_sqls import *
import datetime
import pandas as pd
sys.path.append('../models/')
from ending_onhand import *
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999 

In [13]:
with open("../dev/map.html", "r", encoding='utf-8') as f:
    text= f.read()

In [17]:
eu = Email_Utils()
eu.send_email('[TEST] MAP-DRAWING WH <-> Customer address route (distance, leadtime)'
                ,'INFORMATION'
                ,'''TEST // JUST EXEC HTML FILE IN BROWSER''' 
              , attachment_path='C:/Users/qcells/Desktop/ATP/US_ATP_BI/dev/map.html'
                ,RnRs=['DEV'])

[EVENT] SENT MAIL TO leo.lee@hanwha.com;dany.shin@hanwha.com;joshua.kwak@hanwha.com


In [9]:
with open("../dev/map.html", "r", encoding='utf-8') as f:
    text= f.read()
Txoutlook = eu.outlook.CreateItem(0)
Txoutlook.To = 'dany.shin@hanwha.com' # Destination mail address           
print('[EVENT] SENT MAIL TO {}'.format(str('dany.shin@hanwha.com')))
Txoutlook.Subject = 'OUTBOUND ROUTE DRAW TEST'
Txoutlook.HTMLBody = f"""{text}"""
Txoutlook.Send()

[EVENT] SENT MAIL TO dany.shin@hanwha.com


In [10]:
text

'<!DOCTYPE html>\n<head>    \n    <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n    \n        <script>\n            L_NO_TOUCH = false;\n            L_DISABLE_3D = false;\n        </script>\n    \n    <style>html, body {width: 100%;height: 100%;margin: 0;padding: 0;}</style>\n    <style>#map {position:absolute;top:0;bottom:0;right:0;left:0;}</style>\n    <script src="https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.js"></script>\n    <script src="https://code.jquery.com/jquery-1.12.4.min.js"></script>\n    <script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/js/bootstrap.min.js"></script>\n    <script src="https://cdnjs.cloudflare.com/ajax/libs/Leaflet.awesome-markers/2.0.2/leaflet.awesome-markers.js"></script>\n    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/leaflet@1.6.0/dist/leaflet.css"/>\n    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap.min.css"/>\n    <link rel="stylesheet" href="h

In [2]:
eoh = Ending_On_Hand()

In [18]:
def iso_yyyy(x, cw, rn):
    if rn >= 2:
        if int(cw[-2:]) > 25:
            return df_ending_onhand_result_join[df_ending_onhand_result_join['RANK_YYYY']==2].YYYY_BI.min()
        else:
            return df_ending_onhand_result_join[df_ending_onhand_result_join['RANK_YYYY']==2].YYYY_BI.max()
    else:
        return x
eoh.get_atp_data()
df_ending_onhand_result = pd.DataFrame()
for eoh.ic in pd.concat([eoh.df_atp['Item_Code'], eoh.df_onhand['Item_Code']]).unique():        
    df_union = eoh.get_ending_onhand(eoh.ic)
    df_finals = eoh.calculate_ending_onhand(df_union)
    df_finals = eoh.reshape_ending_onhand(df_finals)
    df_ending_onhand_result = df_ending_onhand_result.append(df_finals)
    break
df_ending_onhand_result_join = eoh.join_ending_onhand(df_ending_onhand_result)
df_ending_onhand_result_join['YYYY_BI'] = df_ending_onhand_result_join['Dates'].apply(lambda x:eoh.get_yyyy(x))
df_ending_onhand_result_join['RANK_YYYY'] = df_ending_onhand_result_join.groupby(['CW'])['YYYY_BI'].transform('nunique')
df_ending_onhand_result_join['YYYY_BI'] = df_ending_onhand_result_join.apply(lambda x: iso_yyyy(x['YYYY_BI'],x['CW'], x['RANK_YYYY']), axis = 1)

In [10]:
df_ending_onhand_result_join['YYYY_BI'] = df_ending_onhand_result_join[df_ending_onhand_result_join['RANK_YYYY']==2].apply(lambda x: eoh.iso_yyyy(x['YYYY_BI'],x['CW']), axis = 1)

In [11]:
df_ending_onhand_result_join.head()

,WH_Location,Item_Code,Dates,CPO_shipped,Hard_Allocation,Intake,ON_HAND,Soft_Allocation,ProductName,Power_Class,SegmentName,Tariff,WestEast,CW,CW_Type,YYYY_BI,RANK_YYYY
0,CA_N,99999973H,2022-07-01,0,0,0,0,0,Q.PEAK DUO BLK ML G10+,400,Residential,0.1475,W,CW26,Reconsil Period,NaN,1
1,CA_N,99999973H,2022-07-02,0,0,0,0,0,Q.PEAK DUO BLK ML G10+,400,Residential,0.1475,W,CW26,Reconsil Period,NaN,1
2,CA_N,99999973H,2022-07-03,0,0,0,0,0,Q.PEAK DUO BLK ML G10+,400,Residential,0.1475,W,CW26,Reconsil Period,NaN,1
3,CA_N,99999973H,2022-07-04,0,0,0,0,0,Q.PEAK DUO BLK ML G10+,400,Residential,0.1475,W,CW27,Reconsil Period,NaN,1
4,CA_N,99999973H,2022-07-05,0,0,0,0,0,Q.PEAK DUO BLK ML G10+,400,Residential,0.1475,W,CW27,Reconsil Period,NaN,1


In [6]:
def get_yyyy(x):
    return str(x[:4])

def iso_yyyy(x, cw):
    if int(cw[-2:]) > 25:
        return df_ending_onhand_result_join[df_ending_onhand_result_join['RANK_YYYY']==2].YYYY_BI.min()
    else:
        return df_ending_onhand_result_join[df_ending_onhand_result_join['RANK_YYYY']==2].YYYY_BI.max()
        

df_ending_onhand_result_join['YYYY_BI'] = df_ending_onhand_result_join['Dates'].apply(lambda x:get_yyyy(x))
df_ending_onhand_result_join['RANK_YYYY'] = df_ending_onhand_result_join.groupby(['CW'])['YYYY_BI'].transform('nunique')
df_ending_onhand_result_join['YYYY_BI'] = df_ending_onhand_result_join[df_ending_onhand_result_join['RANK_YYYY']==2].apply(lambda x: iso_yyyy(x['YYYY_BI'],x['CW']), axis = 1)

In [7]:
df_ending_onhand_result_join[df_ending_onhand_result_join['RANK_YYYY']==2].YYYY_BI.max()

'2022'

In [8]:
df_ending_onhand_result_join[col_db].head()

,WH_Location,Item_Code,ProductName,Power_Class,Tariff,WestEast,Dates,CPO_shipped,Hard_Allocation,Intake,ON_HAND,Soft_Allocation,CW_Type,SegmentName,YYYY_BI
0,CA_N,99999973H,Q.PEAK DUO BLK ML G10+,400,0.1475,W,2022-07-01,0,0,0,0,0,Reconsil Period,Residential,NaN
1,CA_N,99999973H,Q.PEAK DUO BLK ML G10+,400,0.1475,W,2022-07-02,0,0,0,0,0,Reconsil Period,Residential,NaN
2,CA_N,99999973H,Q.PEAK DUO BLK ML G10+,400,0.1475,W,2022-07-03,0,0,0,0,0,Reconsil Period,Residential,NaN
3,CA_N,99999973H,Q.PEAK DUO BLK ML G10+,400,0.1475,W,2022-07-04,0,0,0,0,0,Reconsil Period,Residential,NaN
4,CA_N,99999973H,Q.PEAK DUO BLK ML G10+,400,0.1475,W,2022-07-05,0,0,0,0,0,Reconsil Period,Residential,NaN
